In [ ]:
import build_users_data as build
import projects
import facilities
import users_and_facilities
import users_and_projects

import pandas as pd

In [ ]:
# reporting_periods = ['2013-2014', '2015-2017', '2018-2020', '2021']
# mrr_data_years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
reporting_periods = ["2018-2020"]
mrr_data_years = ["2018", "2019", "2020"]

compliance_report_path = "../data/compliance-reports/"
mrr_data_path = "../data/mrr-data/"
issuance_table_path = "../data/"

In [ ]:
mrr_file_year = {
    "2013": "2019",
    "2014": "2019",
    "2015": "2019",
    "2016": "2020",
    "2017": "2020",
    "2018": "2020",
    "2019": "2020",
    "2020": "2021",
    "2021": "2022",
}
reporting_periods = {
    "2013": "2013-2014",
    "2014": "2013-2014",
    "2015": "2015-2017",
    "2016": "2015-2017",
    "2017": "2015-2017",
    "2018": "2018-2020",
    "2019": "2018-2020",
    "2020": "2018-2020",
    "2021": "2021",
}

In [ ]:
project_df = projects.read_project_data(issuance_table_path)

In [ ]:
facility_df = facilities.read_facility_data(mrr_data_path, mrr_data_years)

In [ ]:
data_path = compliance_report_path

user_project_df = pd.DataFrame()
for reporting_period in reporting_periods:
    df = pd.read_excel(
        data_path + reporting_period + "compliancereport.xlsx",
        sheet_name=reporting_period + " " + "Offset Detail",
        skiprows=4,
    )
    df = df[~pd.isnull(df).any(axis=1)]
    df["reporting_period"] = reporting_period
    user_project_df = user_project_df.append(df)
rename_d = {
    "Entity ID": "user_id",
    "Quantity": "quantity",
    "ARB Project ID #": "arb_id",
}
user_project_df.rename(
    columns=rename_d,
    inplace=True,
)
user_project_df = user_project_df[
    ["user_id", "arb_id", "quantity", "reporting_period"]
]
user_project_df["user_id"] = user_project_df["user_id"].str.strip()
user_project_df["arb_id"] = user_project_df["arb_id"].str.strip()

# ignoring offset vintage lets us simplify arb_id and collapse
# rows that had the same entity and project but different vintages
user_project_df["arb_id"] = (
    user_project_df["arb_id"].str.split("-").apply(lambda x: x[0])
)

user_project_df = (
    user_project_df.groupby(["user_id", "arb_id", "reporting_period"])[
        "quantity"
    ]
    .sum()
    .astype(int)
    .reset_index()
)

In [ ]:
user_project_df

In [ ]:
user_project_df = users_and_projects.read_user_project_data(
    compliance_report_path, reporting_periods
)

In [ ]:
user_facility_df = users_and_facilities.read_user_facility_data(
    compliance_report_path, reporting_periods
)

In [ ]:
user_facility_df

In [ ]:
user_facility_df.loc[user_facility_df["user_id"] == "CA1279"]